In [43]:
import re
import time
import pandas as pd
import numpy as np

import time

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker
from selenium.webdriver.common.action_chains import ActionChains
from collections import Counter
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

In [80]:
url = "https://www.ubereats.com/tw/category/kaohsiung-khh/breakfast-and-brunch"

In [81]:
# 创建 Chrome 选项
chrome_options = webdriver.ChromeOptions()

# 禁用浏览器通知
prefs = {"profile.default_content_setting_values.notifications": 2}
chrome_options.add_experimental_option("prefs", prefs)

# 初始化浏览器驱动，并应用选项
driver = webdriver.Chrome(options=chrome_options)

driver.get(url)

In [82]:
all_restaurant = driver.find_elements(By.CLASS_NAME,'ak.bu')

In [83]:
# 在用class搜索的時候，會搜索到不是目標的element，過濾餐廳
all_restaurant = [r for r in all_restaurant if len(r.get_attribute('class').split())==2]

In [84]:
uber_eat_link_list = [r.find_element(By.TAG_NAME,'a').get_attribute('href') for r in all_restaurant]

In [85]:
name_list = [r.find_element(By.TAG_NAME,'h3').text for r in all_restaurant]

In [86]:
rating_XPath = ".//div/div[2]/div[3]"
rating_list = []
for r in all_restaurant:
    # 有些餐廳是剛加入UberEat，那些餐廳沒有評分，用None代替
    if r.find_element(By.XPATH,rating_XPath).text == '全新':
        rating_list.append(None)
    else:
        rating_list.append(r.find_element(By.XPATH,rating_XPath).text)

In [87]:
len(name_list),len(uber_eat_link_list),len(rating_list)

(80, 80, 80)

In [104]:
df = pd.DataFrame({'name':name_list,'uber_eat_link_list':uber_eat_link_list,'rating':rating_list})

In [105]:
df.head()

,name,uber_eat_link_list,rating
0,呷賀丨麻婆 咖哩 口水雞,https://www.ubereats.com/tw/store/%E5%91%B7%E8...,4.8
1,澳私町餐酒館,https://www.ubereats.com/tw/store/%E6%BE%B3%E7...,4.5
2,熱樂煎爆漿乳酪三明治,https://www.ubereats.com/tw/store/%E7%86%B1%E6...,4.6
3,李子 熱壓吐司 法式吐司 豬米花,https://www.ubereats.com/tw/store/%E6%9D%8E%E5...,4.9
4,貝果 奶油乳酪專賣店,https://www.ubereats.com/tw/store/%E8%B2%9D%E6...,None


In [ ]:
map_driver = webdriver.Chrome(options=chrome_options)
actions=ActionChains(map_driver)

url ='https://www.google.com/maps/?hl=zh-TW&entry=ttu'
comment_btn_xpath = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]'
comment_num_xpath = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[2]/div/div[2]/div[3]'
comment_div_class = 'jftiEf.fontBodyMedium '
address_xpath = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[9]/div[3]/button/div/div[2]/div[1]'
all_comment_list = []
address_list = []

map_driver.get(url)

search_box = map_driver.find_element(By.CLASS_NAME,'fontBodyMedium.searchboxinput.xiQnY ')

for restaurant_name in name_list:
    search_box.clear()
    
    search_box.send_keys(restaurant_name+Keys.ENTER)
    # 有些餐廳沒有在googlemap注冊,要進行判斷
    try:
        WebDriverWait(map_driver, 8).until(
            EC.presence_of_element_located((By.XPATH,address_xpath ))
        )
        address = map_driver.find_element(By.XPATH,address_xpath).text
        address_list.append(address)
        print(address)
    except:
        print(f"{restaurant_name}沒有注冊GoogleMap，沒有評論")
        address_list.append("")
        all_comment_list.append("")
        continue
    
    WebDriverWait(map_driver, 5).until(
        EC.presence_of_element_located((By.XPATH,comment_btn_xpath ))
    )
    # click comment btn
    comment_btn = map_driver.find_element(By.XPATH,comment_btn_xpath)
    comment_btn.click()
    
    # 該餐廳的總評論數
    
    WebDriverWait(map_driver, 10).until(
        EC.presence_of_element_located((By.XPATH, comment_num_xpath))
    )
    
    c = map_driver.find_element(By.XPATH,comment_num_xpath)
    len_comment = int(c.text.split()[0].replace(',',''))
    print(f"{restaurant_name}總評論：{len_comment}")
    
    c.click()
    comment_list = map_driver.find_elements(By.CLASS_NAME,comment_div_class)
    
    # 如果commentlist的長度小於20，滾動頁面
    while len(comment_list) <= 20:
        for i in range(5):
            actions.send_keys(Keys.PAGE_DOWN).perform()
        comment_list = map_driver.find_elements(By.CLASS_NAME,comment_div_class)
        # 如果該餐廳的總評論數小於20,提早結束loop
        if len(comment_list) == len_comment:
            break

    print(f"{restaurant_name}的評論：{len(comment_list)}")


    ## 將comment_list中的comment進行過濾並放入all_comment_list
    temp_list = []
    for comment in comment_list:
        # 留言内容 有些評論只有留下服務的星數等，但是沒有評論，可能導致抓不到span
        try:
            content = comment.find_element(By.CLASS_NAME,'MyEned').find_elements(By.TAG_NAME,'span')
        # 跳過該評論即可 
        except:
            continue
        # 有點擊詳情
        if len(content) == 2:
            content[-1].find_element(By.TAG_NAME,'button').click()
            temp_list.append(content[0].text.replace('\n',''))
        # 無點擊詳情且comment長度大於等於5
        elif len(content[0].text) >= 5:
            temp_list.append(content[0].text.replace('\n',''))
    
    print(f"{restaurant_name}經過過濾後的評論：{len(temp_list)}")
    
    all_comment_list.append(temp_list)

呷賀丨麻婆 咖哩 口水雞沒有注冊GoogleMap，沒有評論
807高雄市三民區民族一路72號
澳私町餐酒館總評論：318
澳私町餐酒館的評論：30
澳私町餐酒館經過過濾後的評論：30
807高雄市三民區合江街11號
熱樂煎爆漿乳酪三明治總評論：2218
熱樂煎爆漿乳酪三明治的評論：30
熱樂煎爆漿乳酪三明治經過過濾後的評論：30
807高雄市三民區吉林街150號
李子 熱壓吐司 法式吐司 豬米花總評論：43
李子 熱壓吐司 法式吐司 豬米花的評論：30
李子 熱壓吐司 法式吐司 豬米花經過過濾後的評論：30
貝果 奶油乳酪專賣店沒有注冊GoogleMap，沒有評論
肆意早午餐.下午茶沒有注冊GoogleMap，沒有評論
801高雄市前金區自立二路117號
MINI.D 自立店總評論：302
MINI.D 自立店的評論：30
MINI.D 自立店經過過濾後的評論：30
802高雄市苓雅區三多四路61號
早到晚到 三多店總評論：101
早到晚到 三多店的評論：30
早到晚到 三多店經過過濾後的評論：30
802高雄市苓雅區林南街6號
巴索托咖啡館總評論：279
巴索托咖啡館的評論：30
巴索托咖啡館經過過濾後的評論：30
802高雄市苓雅區英義街335號
拉亞漢堡 高雄英義店總評論：133
拉亞漢堡 高雄英義店的評論：30
拉亞漢堡 高雄英義店經過過濾後的評論：30
807高雄市三民區吉林街94號1樓
I ‘ s Café總評論：24
I ‘ s Café的評論：24
I ‘ s Café經過過濾後的評論：22
羅斯朵晴天現烘咖啡 忠言店沒有注冊GoogleMap，沒有評論
高雄牛乳大王 創始總店沒有注冊GoogleMap，沒有評論
咬一口飯糰沒有注冊GoogleMap，沒有評論
Kingmauii 金茂宜咖啡 時尚餐盒沒有注冊GoogleMap，沒有評論
Moon L&D 餐廚沒有注冊GoogleMap，沒有評論
Kingmauii金茂宜咖啡 文濱店沒有注冊GoogleMap，沒有評論
807高雄市三民區陽明路65號
早到晚到 高雄陽明店總評論：99
早到晚到 高雄陽明店的評論：30
早到晚到 高雄陽明店經過過濾後的評論：30
朝食暮饗沒有注冊GoogleMap，沒有評論
老瓊紅茶專賣店沒有注冊GoogleMap，沒有評論
802高雄市苓雅區五福一路153-1號15

In [106]:
df['comment'] = all_comment_list
df['address'] = address_list
df.head()

,name,uber_eat_link_list,rating,comment,address
0,呷賀丨麻婆 咖哩 口水雞,https://www.ubereats.com/tw/store/%E5%91%B7%E8...,4.8,,
1,澳私町餐酒館,https://www.ubereats.com/tw/store/%E6%BE%B3%E7...,4.5,"[來這裡聽歌很棒！可惜位子有點少，餐點如果可以跟多元就更完美了！雞肉帕里尼很好吃😋 …, 第...",807高雄市三民區民族一路72號
2,熱樂煎爆漿乳酪三明治,https://www.ubereats.com/tw/store/%E7%86%B1%E6...,4.6,[在坐下來用餐前，先看了一下評價，結果評價意外的差，好奇之下，就入內用餐，公平的整理一下優點...,807高雄市三民區合江街11號
3,李子 熱壓吐司 法式吐司 豬米花,https://www.ubereats.com/tw/store/%E6%9D%8E%E5...,4.9,[蒜香起司👉超推乳酪絲+起司片+玉米+蒜香醬，超涮嘴會回購金沙奶酥👉不推原本以為是加鹹蛋黃，...,807高雄市三民區吉林街150號
4,貝果 奶油乳酪專賣店,https://www.ubereats.com/tw/store/%E8%B2%9D%E6...,None,,


## 評論過濾

In [ ]:
comments_list_test = []
for comment in comment_list:
    # 留言内容
    try:
        # 有些留言是只有指出服務的星星數，但是沒有評論
        content = comment.find_element(By.CLASS_NAME,'MyEned').find_elements(By.TAG_NAME,'span')
    except:
        continue
    # 有點擊詳情
    if len(content) == 2:
        content[-1].find_element(By.TAG_NAME,'button').click()
        comments_list_test.append(content[0].text)
    # 無點擊詳情且comment長度大於等於5
    elif len(content[0].text) >= 5:
        comments_list_test.append(content[0].text)

In [ ]:
df.head()

,name,uber_eat_link_list,rating,comment,address
0,呷賀丨麻婆 咖哩 口水雞,https://www.ubereats.com/tw/store/%E5%91%B7%E8...,4.8,,
1,澳私町餐酒館,https://www.ubereats.com/tw/store/%E6%BE%B3%E7...,4.5,,
2,熱樂煎爆漿乳酪三明治,https://www.ubereats.com/tw/store/%E7%86%B1%E6...,4.6,,
3,李子 熱壓吐司 法式吐司 豬米花,https://www.ubereats.com/tw/store/%E6%9D%8E%E5...,4.9,,
4,貝果 奶油乳酪專賣店,https://www.ubereats.com/tw/store/%E8%B2%9D%E6...,None,,


In [107]:
df = df.drop(df[df['comment'] == ''].index)

In [108]:
df.to_csv('./dataset/restaurant.csv',sep='|')

In [109]:
df

,name,uber_eat_link_list,rating,comment,address
1,澳私町餐酒館,https://www.ubereats.com/tw/store/%E6%BE%B3%E7...,4.5,"[來這裡聽歌很棒！可惜位子有點少，餐點如果可以跟多元就更完美了！雞肉帕里尼很好吃😋 …, 第...",807高雄市三民區民族一路72號
2,熱樂煎爆漿乳酪三明治,https://www.ubereats.com/tw/store/%E7%86%B1%E6...,4.6,[在坐下來用餐前，先看了一下評價，結果評價意外的差，好奇之下，就入內用餐，公平的整理一下優點...,807高雄市三民區合江街11號
3,李子 熱壓吐司 法式吐司 豬米花,https://www.ubereats.com/tw/store/%E6%9D%8E%E5...,4.9,[蒜香起司👉超推乳酪絲+起司片+玉米+蒜香醬，超涮嘴會回購金沙奶酥👉不推原本以為是加鹹蛋黃，...,807高雄市三民區吉林街150號
6,MINI.D 自立店,https://www.ubereats.com/tw/store/minid-%E8%87...,4.9,[很寬敞很舒適的咖啡館～餐點與飲料都很棒很好喝！推薦抹茶與鮮果樂園，各種蛋糕與吐司盒子也非常...,801高雄市前金區自立二路117號
7,早到晚到 三多店,https://www.ubereats.com/tw/store/%E6%97%A9%E5...,5.0,[📍高雄三多四路 早到晚到ㄗㄉㄨㄉ 宵夜食堂 生意超超 ～ 超好Open：17點～ 25點📱...,802高雄市苓雅區三多四路61號
8,巴索托咖啡館,https://www.ubereats.com/tw/store/%E5%B7%B4%E7...,4.9,[可頌好好吃。店內有分一二樓的內用空間，一樓的位置都是用站的。在一樓吃，飲品會便宜15塊，蠻...,802高雄市苓雅區林南街6號
9,拉亞漢堡 高雄英義店,https://www.ubereats.com/tw/store/%E6%8B%89%E4...,4.8,"[85元買了生菜只有碎片、番茄直接不放、完全敷衍的漢堡, 打給店家只跟我說番茄他忘記放了其他...",802高雄市苓雅區英義街335號
10,I ‘ s Café,https://www.ubereats.com/tw/store/i-%E2%80%98-...,4.9,[1.店內氣氛佳很舒適，佈置擺設很用心，內用均是吧檯的設計，店外也有一桌，不管要自己一人享受...,807高雄市三民區吉林街94號1樓
17,早到晚到 高雄陽明店,https://www.ubereats.com/tw/store/%E6%97%A9%E5...,4.9,[比大昌店好吃！炒泡麵沒有水水的加分出餐速度飛快，蘿蔔糕也好吃，豬排有點硬黑輪再酥一點會更好...,807高雄市三民區陽明路65號
20,樂檸漢堡 苓雅五福門市,https://www.ubereats.com/tw/store/%E6%A8%82%E6...,4.9,[生意很好，現點現做，等待時間需抓20~40分鐘，服務很好！漢堡生菜新鮮好吃，牛肉不會太乾扎...,802高雄市苓雅區五福一路153-1號153-2號


## 資料處理

In [15]:
df = pd.read_csv('./dataset/restaurant.csv',sep='|').drop('Unnamed: 0',axis=1)

In [18]:
df.head()

,name,uber_eat_link_list,rating,comment,address
0,澳私町餐酒館,https://www.ubereats.com/tw/store/%E6%BE%B3%E7...,4.5,"['來這裡聽歌很棒！可惜位子有點少，餐點如果可以跟多元就更完美了！雞肉帕里尼很好吃😋 …',...",807高雄市三民區民族一路72號
1,熱樂煎爆漿乳酪三明治,https://www.ubereats.com/tw/store/%E7%86%B1%E6...,4.6,['在坐下來用餐前，先看了一下評價，結果評價意外的差，好奇之下，就入內用餐，公平的整理一下優...,807高雄市三民區合江街11號
2,李子 熱壓吐司 法式吐司 豬米花,https://www.ubereats.com/tw/store/%E6%9D%8E%E5...,4.9,['蒜香起司👉超推乳酪絲+起司片+玉米+蒜香醬，超涮嘴會回購金沙奶酥👉不推原本以為是加鹹蛋黃...,807高雄市三民區吉林街150號
3,MINI.D 自立店,https://www.ubereats.com/tw/store/minid-%E8%87...,4.9,['很寬敞很舒適的咖啡館～餐點與飲料都很棒很好喝！推薦抹茶與鮮果樂園，各種蛋糕與吐司盒子也非...,801高雄市前金區自立二路117號
4,早到晚到 三多店,https://www.ubereats.com/tw/store/%E6%97%A9%E5...,5.0,['📍高雄三多四路 早到晚到ㄗㄉㄨㄉ 宵夜食堂 生意超超 ～ 超好Open：17點～ 25點...,802高雄市苓雅區三多四路61號


In [20]:
ws = CkipWordSegmenter(model="albert-tiny") 
pos = CkipPosTagger(model="albert-tiny")
ner = CkipNerChunker(model="albert-tiny")

def get_word_count_pos_pair(comment,pos):
    result = []
    word_pos_pair = {} # {'很':'Dfa' ... }
    word_count_pos_pair = {} # {'很': ('Dfa', 17),'的': ('DE', 15) .... }
    
    allow_pos = ['VH','VHC']
    
    word_seg = ws([comment])
    word_pos = pos(word_seg)
    
    # 用於記錄評論中出現所有字的詞性
    for word,pos in zip(word_seg[0],word_pos[0]):
        word_pos_pair[word] = pos
        
    # counter 不可以算字典内的東西，字典不會重複
    # 先將word_seg中出現所有的字進行加總，再將結果加上詞性（未刪減）
    counter = Counter(word_seg[0])
    for word,count in counter.most_common():
        word_count_pos_pair[word] = (word_pos_pair[word],count)
        
    for key,values in word_count_pos_pair.items():
        if values[0] in allow_pos and len(key) > 1:
            result.append((key,values[1]))
            
    return result

In [21]:

process_comment = []
for c in df.comment:
    c = ''.join(eval(c))
    process_comment.append(get_word_count_pos_pair(c,pos))
    
df['VH_Tokens'] = process_comment
df.to_csv('./dataset/restaurant_v2.csv',sep='|',index=False)

Inference: 100%|██████████| 1/1 [00:22<00:00, 22.91s/it]


In [22]:
# Get Sentiment
df = pd.read_csv('dataset/restaurant_v2.csv',sep='|')

In [24]:
df.head()

,name,uber_eat_link_list,rating,comment,address,VH_Tokens
0,澳私町餐酒館,https://www.ubereats.com/tw/store/%E6%BE%B3%E7...,4.5,"['來這裡聽歌很棒！可惜位子有點少，餐點如果可以跟多元就更完美了！雞肉帕里尼很好吃😋 …',...",807高雄市三民區民族一路72號,"[('可惜', 5), ('好吃', 4), ('親切', 4), ('明顯', 3), (..."
1,熱樂煎爆漿乳酪三明治,https://www.ubereats.com/tw/store/%E7%86%B1%E6...,4.6,['在坐下來用餐前，先看了一下評價，結果評價意外的差，好奇之下，就入內用餐，公平的整理一下優...,807高雄市三民區合江街11號,"[('好吃', 6), ('不錯', 4), ('特別', 4), ('這樣', 3), (..."
2,李子 熱壓吐司 法式吐司 豬米花,https://www.ubereats.com/tw/store/%E6%9D%8E%E5...,4.9,['蒜香起司👉超推乳酪絲+起司片+玉米+蒜香醬，超涮嘴會回購金沙奶酥👉不推原本以為是加鹹蛋黃...,807高雄市三民區吉林街150號,"[('好吃', 10), ('實在', 6), ('熱壓吐司', 5), ('熱壓', 3)..."
3,MINI.D 自立店,https://www.ubereats.com/tw/store/minid-%E8%87...,4.9,['很寬敞很舒適的咖啡館～餐點與飲料都很棒很好喝！推薦抹茶與鮮果樂園，各種蛋糕與吐司盒子也非...,801高雄市前金區自立二路117號,"[('不錯', 12), ('舒服', 6), ('舒適', 5), ('好吃', 5), ..."
4,早到晚到 三多店,https://www.ubereats.com/tw/store/%E6%97%A9%E5...,5.0,['📍高雄三多四路 早到晚到ㄗㄉㄨㄉ 宵夜食堂 生意超超 ～ 超好Open：17點～ 25點...,802高雄市苓雅區三多四路61號,"[('不錯', 9), ('直接', 7), ('乾淨', 3), ('適合', 3), (..."


In [27]:
# Load Sentiment Model
model = AutoModelForSequenceClassification.from_pretrained("clhuang/albert-sentiment")
# tokenizer
tokenizer = AutoTokenizer.from_pretrained("clhuang/albert-sentiment") #from huggingface
sentiment_classify = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

In [28]:
def get_sentiment_proba(text):
    output = sentiment_classify(text,max_length = 300)
    # positive
    if output[0]['label'] == 'LABEL_1':
        prob_positive = round(output[0]['score'],2)
        prob_negative = round(1 - prob_positive,2)
        
    elif output[0]['label'] == 'LABEL_0':
        prob_negative = round(output[0]['score'],2)
        prob_positive = round(1 - prob_negative,2)
        
    response = {'positive':prob_positive,'negative':prob_negative}
    
    return response

In [31]:
positive_sent_list = []
for i in df.iterrows():
    comment = i[1].comment
    
    sent_list = [get_sentiment_proba(i) for i in eval(comment)]
    positive = round(np.array([i['positive'] for i in sent_list]).mean(),3)
    
    positive_sent_list.append(positive)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [32]:
len(positive_sent_list)

49

In [33]:
df

,name,uber_eat_link_list,rating,comment,address,VH_Tokens
0,澳私町餐酒館,https://www.ubereats.com/tw/store/%E6%BE%B3%E7...,4.5,"['來這裡聽歌很棒！可惜位子有點少，餐點如果可以跟多元就更完美了！雞肉帕里尼很好吃😋 …',...",807高雄市三民區民族一路72號,"[('可惜', 5), ('好吃', 4), ('親切', 4), ('明顯', 3), (..."
1,熱樂煎爆漿乳酪三明治,https://www.ubereats.com/tw/store/%E7%86%B1%E6...,4.6,['在坐下來用餐前，先看了一下評價，結果評價意外的差，好奇之下，就入內用餐，公平的整理一下優...,807高雄市三民區合江街11號,"[('好吃', 6), ('不錯', 4), ('特別', 4), ('這樣', 3), (..."
2,李子 熱壓吐司 法式吐司 豬米花,https://www.ubereats.com/tw/store/%E6%9D%8E%E5...,4.9,['蒜香起司👉超推乳酪絲+起司片+玉米+蒜香醬，超涮嘴會回購金沙奶酥👉不推原本以為是加鹹蛋黃...,807高雄市三民區吉林街150號,"[('好吃', 10), ('實在', 6), ('熱壓吐司', 5), ('熱壓', 3)..."
3,MINI.D 自立店,https://www.ubereats.com/tw/store/minid-%E8%87...,4.9,['很寬敞很舒適的咖啡館～餐點與飲料都很棒很好喝！推薦抹茶與鮮果樂園，各種蛋糕與吐司盒子也非...,801高雄市前金區自立二路117號,"[('不錯', 12), ('舒服', 6), ('舒適', 5), ('好吃', 5), ..."
4,早到晚到 三多店,https://www.ubereats.com/tw/store/%E6%97%A9%E5...,5.0,['📍高雄三多四路 早到晚到ㄗㄉㄨㄉ 宵夜食堂 生意超超 ～ 超好Open：17點～ 25點...,802高雄市苓雅區三多四路61號,"[('不錯', 9), ('直接', 7), ('乾淨', 3), ('適合', 3), (..."
5,巴索托咖啡館,https://www.ubereats.com/tw/store/%E5%B7%B4%E7...,4.9,['可頌好好吃。店內有分一二樓的內用空間，一樓的位置都是用站的。在一樓吃，飲品會便宜15塊，...,802高雄市苓雅區林南街6號,"[('適合', 9), ('特別', 7), ('高雄', 7), ('不錯', 6), (..."
6,拉亞漢堡 高雄英義店,https://www.ubereats.com/tw/store/%E6%8B%89%E4...,4.8,"['85元買了生菜只有碎片、番茄直接不放、完全敷衍的漢堡, 打給店家只跟我說番茄他忘記放了其...",802高雄市苓雅區英義街335號,"[('可以', 7), ('這樣', 4), ('直接', 2), ('難吃', 2), (..."
7,I ‘ s Café,https://www.ubereats.com/tw/store/i-%E2%80%98-...,4.9,['1.店內氣氛佳很舒適，佈置擺設很用心，內用均是吧檯的設計，店外也有一桌，不管要自己一人享...,807高雄市三民區吉林街94號1樓,"[('適合', 5), ('好吃', 4), ('用心', 3), ('不錯', 3), (..."
8,早到晚到 高雄陽明店,https://www.ubereats.com/tw/store/%E6%97%A9%E5...,4.9,['比大昌店好吃！炒泡麵沒有水水的加分出餐速度飛快，蘿蔔糕也好吃，豬排有點硬黑輪再酥一點會更...,807高雄市三民區陽明路65號,"[('不錯', 5), ('好吃', 4), ('直接', 4), ('乾淨', 3), (..."
9,樂檸漢堡 苓雅五福門市,https://www.ubereats.com/tw/store/%E6%A8%82%E6...,4.9,['生意很好，現點現做，等待時間需抓20~40分鐘，服務很好！漢堡生菜新鮮好吃，牛肉不會太乾...,802高雄市苓雅區五福一路153-1號153-2號,"[('好吃', 8), ('舒適', 6), ('新鮮', 4), ('乾淨', 3), (..."


In [44]:
len(positive_sent_list)
df.shape

(49, 6)

In [42]:
for (a,b) in zip([get_sentiment_proba(i) for i in eval(df.iloc[-4,3])],eval(df.iloc[-4,3])):
    print(a,b)

{'positive': 0.97, 'negative': 0.03} 心情不好所以想吃炸物，打電話過去的溝通很順暢，到現場也直接已經做好，店員小哥態度好又帥，心情好了不少~ 身為不能喝氣泡飲的人，覺得可以自選飲料並且有奶茶真的很棒！1號餐女生吃的飽~
{'positive': 0.03, 'negative': 0.97} 剛剛用Uber eat叫 結果來的餐傻爆眼泰式辣醬給我弄一般外面買的辣醬還要15塊雞塊雞腿是我見過最小的看看之前的評論也不只我這樣乾脆別做了  我真的無言⋯⋯
{'positive': 0.05, 'negative': 0.95} 今天先電話訂餐⋯⋯告知要等20分鐘後來過了半小時 同事去現場取餐時竟然告知沒有接到電話？？？我翻了通話記錄根本沒打錯啊⋯請問我是打到平行時空去了嗎？看到評論 八成是先把餐點給別人了吧？
{'positive': 0.03, 'negative': 0.97} 店家出餐流程嚴重有問題， 12月31日晚上6:08致電訂餐，店家表示需要等候30分鐘， 7:10到現場的時候店家餐點沒有完成，甚至將完成的餐點先給Uber和foodpanda外送，詢問店家餐點需要多久才會好店家只會機械化的回覆還要2分鐘，並且態度十分消極，訂餐的體驗非常差，也感受到一家沒有流程的餐廳可以多讓人感到不悅。如果在聚餐的時候不希望被干擾到心情和氣氛的，建議找別家更流程化的店家。
{'positive': 0.4, 'negative': 0.6} 好了不會通知 害我等很久 我就站在門口等超久
{'positive': 0.03, 'negative': 0.97} 前一天在鳳山店買size跟一般正常雞腿差不多今天在高醫店居然買到迷你版？（比iPhone X還短小⋯）看了一下店名確定一樣是胖老爹沒錯🤦🏻‍♂️沒想到品質⋯差⋯這麼多 ⋯難怪一桶只要289⋯吃完整桶確定會飽嗎🤷🏼‍♀️詢問店員該店是站前店還是高醫店，店員不正面回應並說：怎麼了嗎 你們要去評論嗎？原來不是高醫店是維尼店 難怪招牌紅紅的～
{'positive': 0.04, 'negative': 0.96} 雞塊太小，反應店家店家給出的答案是他沒辦法控制，真好笑
{'positive': 0.03, 'negative': 0.97} 等了15分鐘 皮酥脆 有肉汁沒什麼調味味道 不會很油

In [41]:
eval(df.iloc[-4,3])

['心情不好所以想吃炸物，打電話過去的溝通很順暢，到現場也直接已經做好，店員小哥態度好又帥，心情好了不少~ 身為不能喝氣泡飲的人，覺得可以自選飲料並且有奶茶真的很棒！1號餐女生吃的飽~',
 '剛剛用Uber eat叫 結果來的餐傻爆眼泰式辣醬給我弄一般外面買的辣醬還要15塊雞塊雞腿是我見過最小的看看之前的評論也不只我這樣乾脆別做了  我真的無言⋯⋯',
 '今天先電話訂餐⋯⋯告知要等20分鐘後來過了半小時 同事去現場取餐時竟然告知沒有接到電話？？？我翻了通話記錄根本沒打錯啊⋯請問我是打到平行時空去了嗎？看到評論 八成是先把餐點給別人了吧？',
 '店家出餐流程嚴重有問題， 12月31日晚上6:08致電訂餐，店家表示需要等候30分鐘， 7:10到現場的時候店家餐點沒有完成，甚至將完成的餐點先給Uber和foodpanda外送，詢問店家餐點需要多久才會好店家只會機械化的回覆還要2分鐘，並且態度十分消極，訂餐的體驗非常差，也感受到一家沒有流程的餐廳可以多讓人感到不悅。如果在聚餐的時候不希望被干擾到心情和氣氛的，建議找別家更流程化的店家。',
 '好了不會通知 害我等很久 我就站在門口等超久',
 '前一天在鳳山店買size跟一般正常雞腿差不多今天在高醫店居然買到迷你版？（比iPhone X還短小⋯）看了一下店名確定一樣是胖老爹沒錯🤦🏻\u200d♂️沒想到品質⋯差⋯這麼多 ⋯難怪一桶只要289⋯吃完整桶確定會飽嗎🤷🏼\u200d♀️詢問店員該店是站前店還是高醫店，店員不正面回應並說：怎麼了嗎 你們要去評論嗎？原來不是高醫店是維尼店 難怪招牌紅紅的～',
 '雞塊太小，反應店家店家給出的答案是他沒辦法控制，真好笑',
 '等了15分鐘 皮酥脆 有肉汁沒什麼調味味道 不會很油膩但1號餐 兩塊雞塊75元有點貴',
 '服務品質好，衛生做的好，重點雞腿超多汁，很推薦薯條，帶回家冷了也是脆的，超愛的😍😍 …',
 '鮮嫩多汁的🐔，飲料選擇多樣，服務妹子親切有禮.真的需花時間稍候一會兒，但美味的食物值得等待👍ps.每間胖老爹的雞肉是否略有不同?高醫這家感覺新鮮一些欸?',
 '點了外送發現每間品質不一，就屬您這家最糟糕👎雞腿根本縮水一倍也沒有醃入味，薯條也炸很焦 …',
 '食材漏給就算了，腿骨排還有異味臭臭的，跟店員反應要退費，態度不好叫我自己打給老闆退費，最後打給老闆，老闆很客

In [45]:
df['sentimental'] = positive_sent_list

In [47]:
df.to_csv('./dataset/restaurant_v3.csv',sep='|')

In [44]:
df = pd.read_csv('dataset/restaurant_v3.csv',sep='|')
df.iloc[-5:,:]

,Unnamed: 0,name,uber_eat_link_list,rating,comment,address,VH_Tokens,sentimental
44,44,雅米韓式經典料理 高雄總店,https://www.ubereats.com/tw/store/%E9%9B%85%E7...,4.7,['看外送上的照片感覺很好吃結果點了兩碗拌飯都沒有拌飯醬\U0001f972是本來就這樣吃嗎...,807高雄市三民區山東街204號,"[('好吃', 6), ('美味', 4), ('一樣', 4), ('很多', 3), (...",0.497
45,45,胖老爹美式炸雞 高醫店,https://www.ubereats.com/tw/store/%E8%83%96%E8...,4.8,['心情不好所以想吃炸物，打電話過去的溝通很順暢，到現場也直接已經做好，店員小哥態度好又帥，...,807高雄市三民區熱河一街128號,"[('可以', 6), ('好吃', 5), ('糟糕', 4), ('辛苦', 4), (...",0.261
46,46,及第家手作早午餐 大昌店,https://www.ubereats.com/tw/store/%E5%8F%8A%E7...,4.8,['YO蟑螂~又貴又有蟑螂，湯匙一堆都沒洗乾淨...衛生堪憂巧達濃湯=也是牛排店的玉米濃湯這...,807高雄市三民區大昌一路116號,"[('好吃', 5), ('直接', 3), ('不錯', 3), ('乾淨', 2), (...",0.610
47,47,小陀螺激脆可麗餅 三民店,https://www.ubereats.com/tw/store/%E5%B0%8F%E9...,5.0,['大推此店！超脆超好吃的～原本只是意外經過三個人買一份來吃結果隔天直接又來買三份！備註：花...,807高雄市三民區平等路93號,"[('好吃', 2), ('酥脆', 2), ('冷掉', 2), ('超脆', 1), (...",0.920
48,48,GAUTENG COFFEE FREEZE,https://www.ubereats.com/tw/store/gauteng-coff...,NaN,['服務親切，手工麵包口感Q彈有咬勁，橘子香在嘴巴整個散開很好吃。點了評價很高的拿鐵咖啡冰砂...,813高雄市左營區立大路351號,"[('好吃', 11), ('不錯', 7), ('濃郁', 5), ('好喝', 5), ...",0.694


In [12]:
df.sort_values('sentimental').head()

,Unnamed: 0,name,uber_eat_link_list,rating,comment,address,VH_Tokens,sentimental
6,6,拉亞漢堡 高雄英義店,https://www.ubereats.com/tw/store/%E6%8B%89%E4...,4.8,"['85元買了生菜只有碎片、番茄直接不放、完全敷衍的漢堡, 打給店家只跟我說番茄他忘記放了其...",802高雄市苓雅區英義街335號,"[('可以', 7), ('這樣', 4), ('直接', 2), ('難吃', 2), (...",0.098
39,39,御膳坊間,https://www.ubereats.com/tw/store/%E5%BE%A1%E8...,4.8,"['這不會太油嗎？裡面的人都一個臉…去參考一下品味的吧！', '吃完飯剩的油都可以拿來炸年糕...",831高雄市大寮區四維路171號,"[('一樣', 9), ('這樣', 5), ('不錯', 4), ('好吃', 4), (...",0.251
45,45,胖老爹美式炸雞 高醫店,https://www.ubereats.com/tw/store/%E8%83%96%E8...,4.8,['心情不好所以想吃炸物，打電話過去的溝通很順暢，到現場也直接已經做好，店員小哥態度好又帥，...,807高雄市三民區熱河一街128號,"[('可以', 6), ('好吃', 5), ('糟糕', 4), ('辛苦', 4), (...",0.261
36,36,麥當勞 高雄十全 McDonald's S66,https://www.ubereats.com/tw/store/%E9%BA%A5%E7...,4.9,['由於吃早午餐的關係，點了安格斯牛排漢堡套餐、一個炸雞塊。由於套餐都有薯條的關係，所以服務...,807高雄市三民區十全一路197號,"[('這樣', 5), ('好吃', 4), ('直接', 4), ('麻煩', 3), (...",0.358
31,31,禾家冰菓,https://www.ubereats.com/tw/store/%E7%A6%BE%E5...,4.7,['真的驚人，這盒綜合要$90明明單一種口味是$50/盒CP值太低了啦！三種水果份量都不多！...,807高雄市三民區青島街116號,"[('不錯', 3), ('好喝', 3), ('新鮮', 3), ('差勁', 3), (...",0.385


In [29]:
df.iloc[24,:]

Unnamed: 0                                                           24
name                                                          柯制小鮮肉肉蛋吐司
uber_eat_link_list    https://www.ubereats.com/tw/store/%E6%9F%AF%E5...
rating                                                              4.9
comment               ['24.03內用只有騎樓4桌，無空調😔肉蛋吐司的肉嫩味道佳👍肉燥飯看似肥肉很多，但卻一點都...
address                                         804高雄市鼓山區裕誠路No. 1503-8號
VH_Tokens             [('好吃', 8), ('可惜', 6), ('值得', 4), ('簡單', 4), (...
sentimental                                                        0.61
Name: 24, dtype: object

In [45]:
import re

In [46]:
pattern = r"(.{2}區)"
for index,row in df.iterrows():
    district = re.search(pattern=pattern,string=row['address'])
    if district:
        df.loc[index,'district'] = district.group(1)
    else:
        print(row)

In [47]:
df.head()

,Unnamed: 0,name,uber_eat_link_list,rating,comment,address,VH_Tokens,sentimental,district
0,0,澳私町餐酒館,https://www.ubereats.com/tw/store/%E6%BE%B3%E7...,4.5,"['來這裡聽歌很棒！可惜位子有點少，餐點如果可以跟多元就更完美了！雞肉帕里尼很好吃😋 …',...",807高雄市三民區民族一路72號,"[('可惜', 5), ('好吃', 4), ('親切', 4), ('明顯', 3), (...",0.888,三民區
1,1,熱樂煎爆漿乳酪三明治,https://www.ubereats.com/tw/store/%E7%86%B1%E6...,4.6,['在坐下來用餐前，先看了一下評價，結果評價意外的差，好奇之下，就入內用餐，公平的整理一下優...,807高雄市三民區合江街11號,"[('好吃', 6), ('不錯', 4), ('特別', 4), ('這樣', 3), (...",0.542,三民區
2,2,李子 熱壓吐司 法式吐司 豬米花,https://www.ubereats.com/tw/store/%E6%9D%8E%E5...,4.9,['蒜香起司👉超推乳酪絲+起司片+玉米+蒜香醬，超涮嘴會回購金沙奶酥👉不推原本以為是加鹹蛋黃...,807高雄市三民區吉林街150號,"[('好吃', 10), ('實在', 6), ('熱壓吐司', 5), ('熱壓', 3)...",0.629,三民區
3,3,MINI.D 自立店,https://www.ubereats.com/tw/store/minid-%E8%87...,4.9,['很寬敞很舒適的咖啡館～餐點與飲料都很棒很好喝！推薦抹茶與鮮果樂園，各種蛋糕與吐司盒子也非...,801高雄市前金區自立二路117號,"[('不錯', 12), ('舒服', 6), ('舒適', 5), ('好吃', 5), ...",0.799,前金區
4,4,早到晚到 三多店,https://www.ubereats.com/tw/store/%E6%97%A9%E5...,5.0,['📍高雄三多四路 早到晚到ㄗㄉㄨㄉ 宵夜食堂 生意超超 ～ 超好Open：17點～ 25點...,802高雄市苓雅區三多四路61號,"[('不錯', 9), ('直接', 7), ('乾淨', 3), ('適合', 3), (...",0.598,苓雅區


In [48]:
df.to_csv('./dataset/restaurant_v3.csv',sep='|')

In [1]:
import pandas as pd

df = pd.read_csv('./dataset/restaurant_v3.csv',sep='|')

In [2]:
df.district.unique()

array(['三民區', '前金區', '苓雅區', '鹽埕區', '鼓山區', '左營區', '鳳山區', '大寮區'],
      dtype=object)